# Введение в SQL

<img width = '400px' src="images/lesson_10/tds1qm.jpg">

**SQL** (ˈɛsˈkjuˈɛl; англ. structured query language — «язык структурированных запросов») — декларативный язык программирования, применяемый для создания, модификации и управления данными в реляционной базе данных, управляемой соответствующей системой управления базами данных. SQL используется для добавления, обновления и удаления строк данных, извлечения наборов данных для обработки транзакций и аналитических приложений, а также для управления всеми аспектами работы базы данных.

Реляционная база данных – это набор данных с предопределенными связями между ними. Эти данные организованны в виде набора таблиц, состоящих из столбцов и строк. 

In [1]:
import sqlite3 as db
import pandas as pd

СУБД SQLite не обращает внимания на типы данных (она хранит все данные в виде
строк).

Схема работы с БД:
- Подключение к базе данных (вызов connect() с получением объекта - соединения).
- Создание одного или нескольких курсоров (вызов метода объекта-соединения cursor() - CURrrent Set Of Records, текущий набор записей).
- Исполнение команды или запроса (вызов метода execute() или его
вариантов).
- Получение результатов запроса (вызов метода fetchone() или его
вариантов).
- Завершение транзакции или ее откат (вызов метода объекта-соединения
commit() или rollback() ).
- Когда все необходимые транзакции произведены, подключение
закрывается вызовом метода close() объекта-соединения.


In [2]:
import sqlite3
connection = sqlite3.connect("company.db")

cursor = connection.cursor()

# delete 
# cursor.execute("""DROP TABLE employee;""")

sql_command = """
CREATE TABLE employee ( 
staff_number INTEGER PRIMARY KEY, 
fname VARCHAR(20), 
lname VARCHAR(30) DEFAULT 'Ivanov', 
gender CHAR(1), 
joining DATE,
birth_date DATE);"""

cursor.execute(sql_command)

sql_command = """INSERT INTO employee (staff_number, fname, lname, gender, birth_date)
    VALUES (1324, "William", "Shakespeare", "m", "1961-10-25");"""
cursor.execute(sql_command)


sql_command = """UPDATE employee SET fname = 'Ivan' WHERE gender = 'm' """
cursor.execute(sql_command)

sql_command = """DELETE FROM employee WHERE lname = 'Petrov' """
cursor.execute(sql_command)

connection.commit()

connection.close()

In [3]:
connection = sqlite3.connect("company.db")

cursor = connection.cursor()

staff_data = [ ("William", "Shakespeare", "m", "1961-10-25"),
               ("Frank", "Schiller", "m", "1955-08-17"),
               ("Jane", "Wall", "f", "1989-03-14") ]
               
for p in staff_data:
    format_str = """INSERT INTO employee (staff_number, fname, lname, gender, birth_date)
    VALUES (NULL, "{first}", "{last}", "{gender}", "{birthdate}");"""

    sql_command = format_str.format(first=p[0], last=p[1], gender=p[2], birthdate=p[3])
    cursor.execute(sql_command)

connection.commit()

connection.close()    

### CREATE TABLE

Создание базовой таблицы.

CREATE TABLE <имя таблицы>(<список спецификаций столбцов и ограничений>);

In [ ]:
sql_command = """
CREATE TABLE employee ( 
                        staff_number INTEGER PRIMARY KEY, 
                        fname VARCHAR(20), 
                        lname VARCHAR(30) DEFAULT 'Ivanov', 
                        gender CHAR(1), 
                        joining DATE,
                        birth_date DATE                    
                       );""" # PRIMARY KEY есть первичный ключ и оно уникально 

In [39]:
# например есть зарплаты 10, 15, 20, 25 то PRIMARY KEY 1 2 3 4 соответвует

In [40]:
# unique уникально, другого с таким не может быть
# default стандартно


Для обеспечения категорной целостности в языке SQL существуют спецификации **PRIMARY KEY** (первичный ключ) и **UNIQUE** (уникальный ключ). 

### INSERT

Вставка новых записей в таблицу.

INSERT INTO <имя таблицы>[(<имя столбца>,...)]

{VALUES (<значение столбца>,…)}

        | <выражение запроса>     
    
        | {DEFAULT VALUES}

Если отсутствует список столбцов, то список вставляемых значений должен быть полный, то есть обеспечивать значения для всех столбцов таблицы. При этом порядок значений должен соответствовать порядку, заданному оператором CREATE TABLE для таблицы, в которую вставляются строки. 

In [5]:
sql_command = """INSERT INTO employee (staff_number, fname, lname, gender, birth_date)
    VALUES (1324, "William", "Shakespeare", "m", "1961-10-25");"""

In [6]:
sql_command = """INSERT INTO employee
    VALUES (1324, "William", "Shakespeare", "m", "1961-10-25", NULL);"""

In [42]:
sql_command = """INSERT INTO employee
    VALUES (1324, "William", DEFAULT, "m", "1961-10-25", NULL);"""

In [43]:
# узнать есть ли дефолтное, узнать у создателя или в системную

### UPDATE

Изменяет имеющиеся данные в таблице.

UPDATE <имя таблицы>

SET {<имя столбца> = {<выражение для вычисления значения столбца>
                      
        | NULL
                      
        | DEFAULT},...}

        [ {WHERE <предикат>}]

In [8]:
sql_command = """UPDATE employee SET fname = 'Ivan' WHERE gender = 'm' """

### DELETE

Удаляет строки.

DELETE FROM <имя таблицы > 

[WHERE <предикат>];

In [9]:
sql_command = """DELETE FROM employee WHERE lname = 'Petrov' """

### База данных computers

<img width = '500px' src="images/lesson_10/computers.gif">

In [10]:
def sql_query(SQL_QUERY, db):
    connection = sqlite3.connect(db)
    cursor = connection.cursor()
    try:
        if str.lower(SQL_QUERY)[:6] == 'select':
            df = pd.read_sql_query(SQL_QUERY, connection)
            return df

        sql_command = SQL_QUERY
        cursor.execute(sql_command)
    finally:
        connection.commit()
        connection.close()

### Упражнение

Добавить в таблицу Product следующие продукты производителя Z: 
принтер модели 4003, ПК модели 4001 и ноутбук модели 4002.

In [11]:
sql_command = """INSERT INTO product (maker, type, model) VALUES ('Z', 'Printer', 4003), ('Z', 'PC', 4001), ('Z', 'Laptop', 4002)"""
sql_query(sql_command, 'computers.db')

### SELECT

Оператор SELECT осуществляет выборку из базы данных.

есть **LIMIT** - число верхних

In [49]:
sql_query("""SELECT * FROM product """, 'computers.db')

,maker,model,type
0,B,1121,PC
1,A,1232,PC
2,A,1233,PC
3,E,1260,PC
4,A,1276,Printer
5,D,1288,Printer
6,A,1298,Laptop
7,C,1321,Laptop
8,A,1401,Printer
9,A,1408,Printer


In [50]:
sql_query("""SELECT * FROM product LIMIT 5 """, 'computers.db')

,maker,model,type
0,B,1121,PC
1,A,1232,PC
2,A,1233,PC
3,E,1260,PC
4,A,1276,Printer


In [13]:
sql_query("""SELECT model, code FROM pc """, 'computers.db')

,model,code
0,1232,1
1,1121,2
2,1233,3
3,1121,4
4,1121,5
5,1233,6
6,1232,7
7,1232,8
8,1232,9
9,1260,10


**DISTINCT**  - только уникальные значения.

In [14]:
sql_query("""SELECT distinct model FROM pc """, 'computers.db')

,model
0,1232
1,1121
2,1233
3,1260


**ORDER BY** -сортировка запроса по полю.

In [51]:
sql_query("""SELECT distinct model FROM pc ORDER BY 1""", 'computers.db')

,model
0,1121
1,1232
2,1233
3,1260


In [56]:
sql_query("""SELECT model FROM pc ORDER BY model""", 'computers.db')

,model
0,1121
1,1121
2,1121
3,1232
4,1232
5,1232
6,1232
7,1233
8,1233
9,1233


**DESC** - по убыванию

**ASC** - по возрастанию

In [55]:
sql_query("""SELECT distinct model FROM pc ORDER BY model DESC""", 'computers.db')

,model
0,1260
1,1233
2,1232
3,1121


**WHERE** - задает условие на горизонтальную выборку.

In [17]:
sql_query("""SELECT * FROM pc WHERE model = 1260 """, 'computers.db')

,code,model,speed,ram,hd,cd,price
0,10,1260,500,32,10.0,12x,350


<img width = '500px' src="images/lesson_10/imgs_.png">

Переименование столбцов:

In [57]:
sql_query("""SELECT * FROM PC""", 'computers.db')

,code,model,speed,ram,hd,cd,price
0,1,1232,500,64,5.0,12x,600
1,2,1121,750,128,14.0,40x,850
2,3,1233,500,64,5.0,12x,600
3,4,1121,600,128,14.0,40x,850
4,5,1121,600,128,8.0,40x,850
5,6,1233,750,128,20.0,50x,950
6,7,1232,500,32,10.0,12x,400
7,8,1232,450,64,8.0,24x,350
8,9,1232,450,32,10.0,24x,350
9,10,1260,500,32,10.0,12x,350


In [18]:
sql_query("""SELECT ram AS Mb, hd Gb FROM PC WHERE cd = '24x'; """, 'computers.db')

,Mb,Gb
0,64,8.0
1,32,10.0


### Предикаты SQL

**AND, OR, NOT**

In [59]:
# модели, не являющиеся ПК или принтером, т.е. модели ноутбуков в нашем случае
sql_query("""SELECT * FROM Product WHERE NOT (type='PC' OR type='Printer'); """, 'computers.db')

,maker,model,type
0,A,1298,Laptop
1,C,1321,Laptop
2,B,1750,Laptop
3,A,1752,Laptop
4,Z,4002,Laptop
5,Z,4002,Laptop
6,Z,4002,Laptop


**EXISTS/NOT EXISTS**

Предикат EXISTS принимает значение TRUE, если подзапрос содержит любое количество строк, иначе его значение равно FALSE. 

In [20]:
# Найти тех производителей портативных компьютеров, которые также производят принтеры
sql_query("""SELECT DISTINCT maker
FROM Product AS lap_product
WHERE type = 'Laptop' AND 
 EXISTS (SELECT maker
 FROM Product
 WHERE type = 'Printer' AND 
 maker = lap_product.maker
 );
 
""", 'computers.db')

,maker
0,A
1,Z


**BETWEEN**

In [62]:
sql_query("""SELECT model, speed, price FROM PC WHERE price BETWEEN 400 AND 600;""", 'computers.db')

,model,speed,price
0,1232,500,600
1,1233,500,600
2,1232,500,400


**IN**

In [22]:
sql_query("""SELECT model, speed, hd FROM PC WHERE hd IN (10, 20);""", 'computers.db')

,model,speed,hd
0,1233,750,20.0
1,1232,500,10.0
2,1232,450,10.0
3,1260,500,10.0
4,1233,800,20.0


### Упражнение

Найдите номер модели, объем памяти и размеры экранов ПК-блокнотов, цена которых превышает 1000 дол.

In [23]:
sql_query("""SELECT model,ram,screen FROM Laptop WHERE price > 1000""", 'computers.db')

,model,ram,screen
0,1750,128,14
1,1298,64,15
2,1752,128,14


Найдите номер модели, скорость и размер жесткого диска ПК, имеющих 12x или 24x CD и цену менее 600 дол.

In [76]:
sql_query("""SELECT model, speed, hd FROM pc WHERE cd IN ('12x', '24x') AND price < 600""", 'computers.db')

,model,speed,hd
0,1232,500,10.0
1,1232,450,8.0
2,1232,450,10.0
3,1260,500,10.0


**LIKE**

Символ (_) -  вместо любого единичного символа в проверяемом значении;

Символ (%) - заменяет последовательность любых символов (число символов в последовательности может быть от 0 и более) в проверяемом значении.

<img width = '700px' src="images/lesson_10/imgs_2.png">

In [77]:
sql_query("""SELECT * FROM product WHERE type LIKE 'P%'  """, 'computers.db')

,maker,model,type
0,B,1121,PC
1,A,1232,PC
2,A,1233,PC
3,E,1260,PC
4,A,1276,Printer
5,D,1288,Printer
6,A,1401,Printer
7,A,1408,Printer
8,D,1433,Printer
9,E,1434,Printer


In [25]:
sql_query("""SELECT * FROM product WHERE type LIKE 'P%' and type not like 'PC%'  """, 'computers.db')

,maker,model,type
0,A,1276,Printer
1,D,1288,Printer
2,A,1401,Printer
3,A,1408,Printer
4,D,1433,Printer
5,E,1434,Printer
6,Z,4003,Printer
7,Z,4003,Printer
8,Z,4003,Printer


In [26]:
sql_query("""SELECT * FROM product WHERE model LIKE '12%' """, 'computers.db')

,maker,model,type
0,A,1232,PC
1,A,1233,PC
2,E,1260,PC
3,A,1276,Printer
4,D,1288,Printer
5,A,1298,Laptop


### Упражнение

Найди все модели принтеров, которые начинаюстя на 1 и заканчиваются на 1.

In [81]:
sql_query("""SELECT * FROM printer WHERE model LIKE '1%1'""", 'computers.db')

,code,model,color,type,price
0,4,1401,n,Matrix,150


### Агрегаты

**SUM, MIN, MAX, AVG, COUNT** 

In [27]:
sql_query("""SELECT MIN(price) AS Min_price, MAX(price) AS Max_price FROM PC; """, 'computers.db')

,Min_price,Max_price
0,350,980


### Упражнение

Найти имеющееся в наличии количество компьютеров, выпущенных производителем А.

In [96]:
sql_query("""SELECT COUNT(*) AS n FROM pc
WHERE model IN (SELECT model FROM product WHERE maker='A')""", 'computers.db')

,n
0,8


Найти количество имеющихся различных моделей ПК, выпускаемых производителем А.

In [102]:
sql_query("""SELECT COUNT(distinct model) AS n FROM pc
WHERE model IN (SELECT model FROM product WHERE maker='A')""", 'computers.db')

,n
0,2


**GROUP BY/HAVING**

<img width = '700px' src="images/lesson_10/sql-having.png">

**GROUP BY** используется для объединения результатов выборки по одному или нескольким столбцам.

In [28]:
sql_query("""SELECT model, COUNT(model) AS Qty_model, 
    AVG(price) AS Avg_price
FROM PC
GROUP BY model; """, 'computers.db')

,model,Qty_model,Avg_price
0,1121,3,850.0
1,1232,4,425.0
2,1233,4,875.0
3,1260,1,350.0


Если предложение WHERE определяет предикат для фильтрации строк, то предложение **HAVING** применяется после группировки для определения аналогичного предиката, фильтрующего группы по значениям агрегатных функций. 

In [29]:
sql_query("""SELECT model, COUNT(model) AS Qty_model, 
    AVG(price) AS Avg_price
FROM PC
GROUP BY model
HAVING avg(price) > 500
; """, 'computers.db')

,model,Qty_model,Avg_price
0,1121,3,850.0
1,1233,4,875.0


Правильный порядок:
    
**SELECT** <выражение>

**FROM** <имя таблицы> 

[**WHERE** <предикат>]

[[**GROUP BY** <список столбцов>]

[**HAVING** <условие на агрегатные значения>] ]

[**ORDER BY** <список столбцов>]

### Упражнение

Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.

In [113]:
sql_query("""SELECT maker, COUNT(model) AS cnt FROM product WHERE type='PC' 
GROUP BY maker HAVING COUNT(DISTINCT model) >= 3""", 'computers.db')

,maker,cnt
0,E,3


Найдите средний размер диска ПК (одно значение для всех) тех производителей, которые выпускают и принтеры. Вывести: средний размер HD

In [124]:
sql_query("""SELECT AVG(hd) FROM pc, product 
WHERE product.model = pc.model AND product.maker IN 
(SELECT DISTINCT maker FROM product WHERE product.type = 'Printer')""", 'computers.db')

,AVG(hd)
0,14.222222


In [125]:
sql_query("""SELECT AVG(hd) FROM pc WHERE model IN(SELECT model 
    FROM product WHERE 
        maker IN(SELECT maker FROM product WHERE type = 'Printer'))""", 'computers.db')

,AVG(hd)
0,14.222222


### Выбор из нескольких источников.

In [30]:
sql_query("""SELECT *
FROM product
where type = 'Printer'
 """, 'computers.db')

,maker,model,type
0,A,1276,Printer
1,D,1288,Printer
2,A,1401,Printer
3,A,1408,Printer
4,D,1433,Printer
5,E,1434,Printer
6,Z,4003,Printer
7,Z,4003,Printer
8,Z,4003,Printer


In [127]:
sql_query("""SELECT distinct type as type_of_printer FROM printer""", 'computers.db')

,type_of_printer
0,Laser
1,Jet
2,Matrix


In [131]:
#Декартово произведение
sql_query("""SELECT *
FROM product, (SELECT distinct type as type_of_printer FROM printer)
where type = 'Printer'
 """, 'computers.db')

,maker,model,type,type_of_printer
0,A,1276,Printer,Laser
1,A,1401,Printer,Laser
2,A,1408,Printer,Laser
3,D,1288,Printer,Laser
4,D,1433,Printer,Laser
5,E,1434,Printer,Laser
6,Z,4003,Printer,Laser
7,Z,4003,Printer,Laser
8,Z,4003,Printer,Laser
9,A,1276,Printer,Jet


<img width = '700px' src="images/lesson_10/joins.png">

**FROM** TABLE_1 **<INNER/RIGHT/LEFT/FULL>** **JOIN** TABLE_2 **ON** ключи

Найти производителя, номер модели и цену каждого компьютера, имеющегося в базе данных:

In [32]:
sql_query("""SELECT maker, Product.model AS model_1, 
            PC.model AS model_2, price
FROM Product INNER JOIN 
 PC ON PC.model = Product.model
ORDER BY maker, model_2;
 """, 'computers.db')

,maker,model_1,model_2,price
0,A,1232,1232,350
1,A,1232,1232,350
2,A,1232,1232,400
3,A,1232,1232,600
4,A,1233,1233,600
5,A,1233,1233,950
6,A,1233,1233,970
7,A,1233,1233,980
8,B,1121,1121,850
9,B,1121,1121,850


In [33]:
sql_query("""SELECT maker, Product.model AS model_1, 
            PC.model AS model_2, price
FROM Product LEFT JOIN 
 PC ON PC.model = Product.model
ORDER BY maker, model_2;
 """, 'computers.db')

,maker,model_1,model_2,price
0,A,1276,None,NaN
1,A,1298,None,NaN
2,A,1401,None,NaN
3,A,1408,None,NaN
4,A,1752,None,NaN
5,A,1232,1232,350.0
6,A,1232,1232,350.0
7,A,1232,1232,400.0
8,A,1232,1232,600.0
9,A,1233,1233,600.0


### Упражнение

Найдите производителей, которые производили бы как ПК со скоростью не менее 750 МГц, так и ПК-блокноты со скоростью не менее 750 МГц. 

In [147]:
sql_query("""SELECT distinct product.maker 
FROM product LEFT JOIN pc ON product.model = pc.model LEFT JOIN laptop ON product.model = laptop.model

""", 'computers.db')

,maker
0,B
1,A
2,E
3,D
4,C
5,Z


In [151]:
sql_query("""SELECT distinct product.maker 
FROM product LEFT JOIN pc ON product.model = pc.model LEFT JOIN laptop ON product.model = laptop.model
where pc.speed >= 750 OR laptop.speed >= 750
""", 'computers.db')

,maker
0,B
1,A


In [157]:
sql_query("""SELECT maker
 FROM Product 
 LEFT JOIN PC ON PC.model = Product.model
 LEFT JOIN LAPTOP ON LAPTOP.model = Product.model
 GROUP BY maker HAVING ((count(PC.speed >= 750) > 0) AND (count(LAPTOP.speed >= 750) > 0 ))
 """, 'computers.db')

,maker
0,A
1,B


**UNION**

Предложение **UNION** приводит к появлению в результирующем наборе всех строк каждого из запросов. При этом, если определен параметр **ALL**, то сохраняются все дубликаты выходных строк, в противном случае в результирующем наборе присутствуют только уникальные строки.

Операция объединения может быть выполнена только при выполнении следующих условий:

- количество выходных столбцов каждого из запросов должно быть одинаковым;

- выходные столбцы каждого из запросов должны быть совместимы между собой (в порядке их следования) по типам данных;

- в результирующем наборе используются имена столбцов, заданные в первом запросе;

Найти номера моделей и цены ПК и портативных компьютеров:

In [34]:
sql_query("""SELECT model, price
FROM PC
UNION
SELECT model, price
FROM Laptop
ORDER BY price DESC;
 """, 'computers.db')

,model,price
0,1750,1200
1,1752,1150
2,1298,1050
3,1233,980
4,1233,970
5,1321,970
6,1233,950
7,1298,950
8,1121,850
9,1298,700


### Упражнение

Найти тип продукции, номер модели и цену ПК и портативных компьютеров:

In [163]:
sql_query("""SELECT 'PC' as type, model, price FROM pc
UNION
SELECT 'LAPTOP' as type, model, price FROM laptop
ORDER BY price""", 'computers.db')

,type,model,price
0,PC,1232,350
1,PC,1260,350
2,PC,1232,400
3,PC,1232,600
4,PC,1233,600
5,LAPTOP,1298,700
6,PC,1121,850
7,LAPTOP,1298,950
8,PC,1233,950
9,LAPTOP,1321,970


In [173]:
sql_query("""SELECT product.type, pc.model as model, pc.price as price FROM product 
    INNER JOIN pc ON product.model = pc.model
UNION
    SELECT product.type, laptop.model as model, laptop.price as price FROM product 
    INNER JOIN laptop ON product.model = laptop.model
WHERE product.type = 'PC' OR product.type = 'Laptop'
ORDER BY price""", 'computers.db')

,type,model,price
0,PC,1232,350
1,PC,1260,350
2,PC,1232,400
3,PC,1232,600
4,PC,1233,600
5,Laptop,1298,700
6,PC,1121,850
7,Laptop,1298,950
8,PC,1233,950
9,Laptop,1321,970


In [172]:
sql_query("""SELECT Product.type, PC.model, price
FROM PC INNER JOIN   
     Product ON PC.model = Product.model
UNION
SELECT Product.type, Laptop.model, price 
FROM Laptop INNER JOIN   
     Product ON Laptop.model = Product.model
ORDER BY price DESC;
 """, 'computers.db')

,type,model,price
0,Laptop,1750,1200
1,Laptop,1752,1150
2,Laptop,1298,1050
3,PC,1233,980
4,Laptop,1321,970
5,PC,1233,970
6,Laptop,1298,950
7,PC,1233,950
8,PC,1121,850
9,Laptop,1298,700


**UNION ALL**

Найти все имеющиеся единицы продукции производителя 'B'. Вывести номер модели и тип.

In [35]:
sql_query("""SELECT p.model, p.type FROM pc JOIN Product p ON PC.model=p.model WHERE maker='B'
UNION ALL
SELECT p.model, p.type FROM printer pr JOIN Product p ON pr.model=p.model WHERE maker='B'
UNION ALL
SELECT p.model, p.type FROM laptop lp JOIN Product p ON lp.model=p.model WHERE maker='B';
 """, 'computers.db')

,model,type
0,1121,PC
1,1121,PC
2,1121,PC
3,1750,Laptop


In [36]:
sql_query("""SELECT p.model, p.type FROM pc JOIN Product p ON PC.model=p.model WHERE maker='B'
UNION 
SELECT p.model, p.type FROM printer pr JOIN Product p ON pr.model=p.model WHERE maker='B'
UNION
SELECT p.model, p.type FROM laptop lp JOIN Product p ON lp.model=p.model WHERE maker='B';
 """, 'computers.db')

,model,type
0,1121,PC
1,1750,Laptop


### Упражнение

Перечислите номера моделей любых типов, имеющих самую высокую цену по всей имеющейся в базе данных продукции.

Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker

Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена.

In [217]:
sql_query("""SELECT AVG(price) as pr FROM(
    SELECT pc.price FROM pc INNER JOIN product 
        ON pc.model = product.model 
    WHERE maker = 'A'
UNION all
    SELECT laptop.price FROM laptop INNER JOIN product 
         ON laptop.model = product.model
    WHERE maker = 'A'
)""", 'computers.db')


,pr
0,754.166667


In [221]:
sql_query("""SELECT AVG(price) as pr 
    FROM (SELECT price, 'PC' as type FROM pc INNER JOIN product 
        ON pc.model = product.model 
    WHERE maker = 'A'
UNION
     SELECT price, 'LP' as type FROM laptop INNER JOIN product 
         ON laptop.model = product.model
    WHERE maker = 'A'
)""", 'computers.db')

,pr
0,810.0


In [191]:
sql_query("""SELECT pc.price, pc.model, product.maker FROM pc INNER JOIN product ON pc.model = product.model 
""", 'computers.db')

,price,model,maker
0,600,1232,A
1,850,1121,B
2,600,1233,A
3,850,1121,B
4,850,1121,B
5,950,1233,A
6,400,1232,A
7,350,1232,A
8,350,1232,A
9,350,1260,E


In [194]:
sql_query("""SELECT pc.price, pc.model, product.maker FROM pc INNER JOIN product ON pc.model = product.model 
UNION 
     SELECT laptop.price, laptop.model, product.maker FROM laptop INNER JOIN product ON laptop.model = product.model
""", 'computers.db')

,price,model,maker
0,350,1232,A
1,350,1260,E
2,400,1232,A
3,600,1232,A
4,600,1233,A
5,700,1298,A
6,850,1121,B
7,950,1233,A
8,950,1298,A
9,970,1233,A


In [207]:
sql_query("""SELECT (SUM(pc.price)*1.0+SUM(laptop.price))/(COUNT(pc.price) + COUNT(laptop.price)) as pr 
FROM product 
    LEFT JOIN pc ON product.model = pc.model 
    LEFT JOIN laptop ON laptop.model = product.model
WHERE maker = 'A'""", 'computers.db')

,pr
0,754.166667


In [216]:
sql_query("""
SELECT AVG(price) as pr FROM(
    SELECT pc.price FROM pc INNER JOIN product 
        ON pc.model = product.model 
    WHERE maker = 'A'
UNION all
    SELECT laptop.price FROM laptop INNER JOIN product 
         ON laptop.model = product.model
    WHERE maker = 'A'
)""", 'computers.db')

